### --- Day 16: Reindeer Maze ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/16

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 16);
WriteLines(inputLines);

Loading puzzle file: Day16.txt
Total lines: 141
Max line length: 141

#############################################################################################################################################
#.............#...#...........#...#...................#.........#...........#...................................#.............#.......#....E#
#.#####.#####.#.#.#.###.#####.#.#.#.#.#########.#####.#.#######.#.#########.#######.###.###.###.#.#######.#.###.#.#########.#.#####.#.#.#.#.#
#.#...#...#.#...#.#...#.....#.#.#.#.#.....#.....#...#...#.......#...#.....#.....#...#.#.....#.#.#.#.....#.#...#.#.#.#.......#...#...#.#.#.#.#
###.#.#.#.#.###.#.###.#.###.#.#.#.#.#.###.#.#####.#######.#########.###.#######.#.###.#######.#.#.#####.#.###.#.#.#.#.#########.#.###.#.#.#.#


In [4]:
string[] testInputLines = [
    "###############",
    "#.......#....E#",
    "#.#.###.#.###.#",
    "#.....#.#...#.#",
    "#.###.#####.#.#",
    "#.#.#.......#.#",
    "#.#.#####.###.#",
    "#...........#.#",
    "###.#.#####.#.#",
    "#...#.....#.#.#",
    "#.#.#.###.#.#.#",
    "#.....#...#.#.#",
    "#.###.#.#.#.#.#",
    "#S..#.....#...#",
    "###############",
];

This looks like a fairly straightforward use of Dijkstra's algorithm, modelling each "node" as a point + direction.

In [5]:
using PointDir = (Point point, Point dir);
using Cost = uint;

In [6]:
using PointDirCost = (PointDir pointDir, Cost cost);
using NextMoveFunc = NextNodeFunc<PointDir, Cost>;

In [7]:
int FindLowestScore(string[] inputLines)
{
    CharGrid grid = new(inputLines);

    var ((end, _), (start, _)) = grid.Enumerate().Where(pch => pch.ch is START or END).OrderBy(pch => pch.ch).ToArray();

    var nextMoveFunc = GetNextMoveFunc(grid);

    var minCost = ShortestPath((start, Right), nextMoveFunc)
                    .First<PointDirCost>(pdc => pdc.pointDir.point == end).cost;
    return (int)minCost;
}

NextMoveFunc GetNextMoveFunc(CharGrid grid)
{
    // They can move forward one tile at a time (increasing their score by 1
    // point), but never into a wall (#). They can also rotate clockwise or
    // counterclockwise 90 degrees at a time (increasing their score by 1000
    // points).
    IEnumerable<PointDirCost> nextMoveFunc(PointDir pointDir, Cost cost)
    {
        var (point, dir) = pointDir;

        var nextStep = point + dir;
        if (grid.IsValid(nextStep) && grid[nextStep] is not WALL) {
            yield return ((nextStep, dir), cost + 1);
        }
        dir = TurnRight(dir);
        yield return ((point, dir), cost + 1000);
        dir = TurnRight(TurnRight(dir));
        yield return ((point, dir), cost + 1000);
    }
    return nextMoveFunc;
}

const char WALL = '#';
const char START = 'S';
const char END = 'E';

In [8]:
// There are many paths through this maze, but taking any of the best paths
// would incur a score of only 7036

var testAnswer = FindLowestScore(testInputLines);
Console.WriteLine(testAnswer);

7036


In [9]:
// Analyze your map carefully. What is the lowest score a Reindeer could
// possibly get?

var part1Answer = FindLowestScore(inputLines);
Console.WriteLine(part1Answer);

94436


In [10]:
// 94436 is correct!
Ensure(94436, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/16

Ok, so in order to figure out this one, we need to know which tiles are on the best path.

For a given tile, we know the previous step that has given the current tile its lowest score. So if we keep track of the "parent" steps, we can trace our way back up the best path. Once we know the path, counting the points along this path is straightforward.

In [12]:
// The cost dictionary to track the parent relationship. Note there could be
// multiple parents if different paths have the same cost, so we'll manage that
// accordingly below

using CostDict = SCG.Dictionary<PointDir, (Cost cost, SCG.List<PointDir> parents)>;

In [13]:
int CountTilesOnBestPath(string[] inputLines, uint knownMinCost)
{
    CharGrid grid = new(inputLines);

    var ((end, _), (start, _)) = grid.Enumerate().Where(pch => pch.ch is START or END).OrderBy(pch => pch.ch).ToArray();

    CostDict costDict = new();
    costDict[(start, Right)] = (0, new());
    var nextMoveFunc = GetNextMoveFunc2(grid, costDict);

    var _ = ShortestPath((start, Right), nextMoveFunc).Count();

    var ends = costDict.Keys.Where(k => k.point == end && costDict[k].cost == knownMinCost);

    var pointCount = ends.SelectMany(end => GetPointPath(costDict, end)).Distinct().Count();
    return pointCount;
}

NextMoveFunc GetNextMoveFunc2(CharGrid grid, CostDict costDict)
{
    // During this exploration phase, we know the reachable points have this
    // current point as their parent. So we can use this to build the cost / parent
    // relationship
    IEnumerable<PointDirCost> nextMoveFunc(PointDir pointDir, Cost cost)
    {
        var (point, dir) = pointDir;

        var nextStep = point + dir;
        if (grid.IsValid(nextStep) && grid[nextStep] is not WALL) {
            PointDirCost nextStepCost = ((nextStep, dir), cost + 1);
            TrySetLowest(nextStepCost, pointDir);
            yield return nextStepCost;
        }
        
        dir = TurnRight(dir);
        PointDirCost nextTurnCost = ((point, dir), cost + 1000);
        TrySetLowest(nextTurnCost, pointDir);
        yield return nextTurnCost;

        dir = TurnRight(TurnRight(dir));
        nextTurnCost = ((point, dir), cost + 1000);
        TrySetLowest(nextTurnCost, pointDir);
        yield return nextTurnCost;
    }
    return nextMoveFunc;

    void TrySetLowest(PointDirCost pdc, PointDir parent)
    {
        var currentCostParent = costDict.GetValueOrDefault(pdc.pointDir, (pdc.cost, new()));

        if (pdc.cost == currentCostParent.cost)
        {
            // Equal-best cost. Add this parent to the existing list.
            currentCostParent.parents.Add(parent);
        }
        else if (pdc.cost < currentCostParent.cost)
        {
            // New best cost with new singular parent.
            currentCostParent = (pdc.cost, [parent]);
        }
        costDict[pdc.pointDir] = currentCostParent;
    }
}

IEnumerable<Point> GetPointPath(CostDict costDict, PointDir current)
{
    var parentPath = from parent in costDict[current].parents
                     from parentPoint in GetPointPath(costDict, parent)
                     select parentPoint;
    return parentPath.Append(current.point);
}
                                                                         

In [14]:
// In the first example, there are 45 tiles (marked O) that are part of at least
// one of the various best paths through the maze:

var part2TestAnswer = CountTilesOnBestPath(testInputLines, 7036);
Console.WriteLine(part2TestAnswer);

45


In [15]:
// Analyze your map further. How many tiles are part of at least one of the best
// paths through the maze?

var part2Answer = CountTilesOnBestPath(inputLines, 94436);
Console.WriteLine(part2Answer);

481


In [16]:
// 481 is correct!
Ensure(481, part2Answer);